# Workflow med Omträning

## Identifiera Svåra Bilder för Omträning
Lägg till felklassificerade bilder i en lista och kopiera dem till en ny mapp för omträning.

In [ ]:
import os
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import psutil

# Kolla batteristatus
battery = psutil.sensors_battery()
if battery and battery.power_plugged is False:
    print("⚠️ Warning: Your laptop is running on battery. Performance may be reduced, "
          "and CUDA might fail due to power-saving features.")

# Check CUDA availability
if torch.cuda.is_available():
    print(f"✅ CUDA is available: Running on {torch.cuda.get_device_name(0)}")
else:
    print("❌ CUDA is not available: Running on CPU. Ensure your GPU drivers are installed correctly.")

# Define paths
train_fire_dir = "dataset/train/fire"
train_non_fire_dir = "dataset/train/non_fire"
retrain_fire_dir = "dataset/retrain/fire"
retrain_non_fire_dir = "dataset/retrain/non_fire"

# ✅ Define transformation (same as before)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Ladda in data från olika kataloger

In [ ]:
# ✅ Custom Dataset to Load from Multiple Directories
class FireDataset(Dataset):
    def __init__(self, fire_dirs, non_fire_dirs, transform=None):
        self.data = []
        self.transform = transform

        # Load images from all fire directories
        for fire_dir in fire_dirs:
            for img_file in os.listdir(fire_dir):
                img_path = os.path.join(fire_dir, img_file)
                self.data.append((img_path, 1))  # Fire = 1

        # Load images from all non-fire directories
        for non_fire_dir in non_fire_dirs:
            for img_file in os.listdir(non_fire_dir):
                img_path = os.path.join(non_fire_dir, img_file)
                self.data.append((img_path, 0))  # Non-Fire = 0

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# ✅ Load dataset dynamically from both directories
fire_dirs = [train_fire_dir, retrain_fire_dir]
non_fire_dirs = [train_non_fire_dir, retrain_non_fire_dir]

train_dataset = FireDataset(fire_dirs, non_fire_dirs, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

print(f"🔄 Training dataset now includes {len(train_dataset)} images (incl. retraining data)")

## Ladda In Data För Omträning
Vi laddar in originaldatasetet plus de extra svåra bilderna.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# ✅ Load MobileNetV2 model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = models.MobileNet_V2_Weights.DEFAULT
model = models.mobilenet_v2(weights=weights)
model.classifier[1] = nn.Linear(model.last_channel, 2)
model.to(device)

# ✅ Define loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ✅ Retrain the model with expanded dataset
num_epochs_retrain = 10
for epoch in range(num_epochs_retrain):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs_retrain}], Loss: {running_loss:.4f}, Train Acc: {train_acc:.2f}%")

# ✅ Save the retrained model
torch.save(model.state_dict(), "fire_classifier_retrained.pth")
print("✅ Retraining complete! Model saved as 'fire_classifier_retrained.pth'.")


## Validation & Testing


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Load test dataset
test_dirs = {
    "fire": "dataset/test/fire",
    "non_fire": "dataset/test/non_fire"
}

true_labels, pred_labels = [], []

# Perform inference
model.eval()
for label, folder in test_dirs.items():
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        image = Image.open(img_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(image)
            probabilities = torch.nn.functional.softmax(output, dim=1)
            _, predicted_class = torch.max(probabilities, 1)

        true_labels.append(1 if label == "fire" else 0)
        pred_labels.append(predicted_class.item())

# Compute confusion matrix
cm = confusion_matrix(true_labels, pred_labels)
class_names = ["non_fire", "fire"]

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix: Retrained Model")
plt.savefig("confusion_matrix_retrained.png")
print("📊 New confusion matrix saved as 'confusion_matrix_retrained.png'.")
